Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [183]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [184]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [185]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [186]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [187]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [188]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}

In [189]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

## Oversimplified match

In [190]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=2)
INFO:root:status: <1 3 5 5 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 2 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 4 2 9>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 2 4 2 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 2 1 2 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 2 1 1 9>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 0 1 1 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 0 0 1 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:status: <1 0 0 1 7>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <1 0 0 0 7>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=7)
INFO:root:status: <1 0

## Task 2.1: Fixed-rules based on nim-sum

For the first task I developed an agent that searches for a move that results in a non-zero nim-sum. If that is not possible, it picks randomly instead.

In [191]:
def nim_sum_agent(state: Nim) -> Nimply:
    for row, count in enumerate(state.rows):
        for objects in range(1, count + 1):
            tmp = deepcopy(state)
            tmp.nimming(Nimply(row, objects))
            if nim_sum(tmp) != 0:
                return Nimply(row, objects)
            
    return pure_random(state)

In [192]:
def compare_strategies(strat_1, strat_2, runs=20):
    counts = [0, 0]
    for _ in range(runs):
        strategy = (strat_1, strat_2)
        nim = Nim(5)
        player = 0
        while nim:
            ply = strategy[player](nim)
            nim.nimming(ply)
            player = 1 - player
        counts[player] += 1
    return counts

We can see that it tends to preform better than random and gabriele, but still struggles against optimal, as they are equivalent.

In [193]:
print(compare_strategies(nim_sum_agent, pure_random))
print(compare_strategies(nim_sum_agent, gabriele))
print(compare_strategies(nim_sum_agent, optimal))

[14, 6]
[20, 0]
[4, 16]


## Task 2.2: An agent using evolved rules using ES

In [194]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

In [195]:
# Create a population of individuals that represent a set of rules for playing nim.

from dataclasses import dataclass


STRATEGIES = [pure_random, gabriele, nim_sum_agent]


@dataclass
class Individual:
    genotype: list
    fitness: None

    def __str__(self) -> str:
        return f"Genotype: {self.genotype}, Fitness: {self.fitness}"

    def pick_strategy(self) -> callable:
        chosen_strategy = random.choices(STRATEGIES, self.genotype)[0]
        return chosen_strategy



# Weights initialized to sum up to 1, representing how likely a strategy is
# to be chosen.
def initialize_population() -> list[Individual]:
    population = []
    for _ in range(POPULATION_SIZE):
        weights = [random.random() for _ in range(len(STRATEGIES))]
        weights = [*[x / sum(weights) for x in weights]]
        population.append(Individual(genotype=weights, fitness=None))

    return population

In [196]:
# Evaluate the fitness of each individual by playing a number of games and counting the wins


def fitness(individual: Individual, num_games=20) -> float:
    wins = 0

    for _ in range(num_games):
        n = Nim(5)
        player = 0
        while n:
            strategy = individual.pick_strategy() if player == 0 else pure_random
            ply = strategy(n)
            n.nimming(ply)
            player = 1 - player

        # opponent picked last
        if player == 1:
            wins += 1

    return wins / num_games


def initialize_fitness(population):
    for p in population:
        p.fitness = fitness(p, num_games=20)

In [197]:
# select individuals to be the next parents
def select_parent(population):
    pool = [random.choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

In [198]:
# mutation and crossover


def mutation(individual: Individual) -> list:
    where = random.randint(0, len(individual.genotype) - 1)
    new = deepcopy(individual.genotype)
    new[where] = random.uniform(0, 1)
    return Individual(genotype=[*(x / sum(new) for x in new)], fitness=None)


def xover(first, second):
    cut_point = random.randint(0, len(STRATEGIES) - 1)
    offspring = Individual(
        fitness=None, genotype=first.genotype[:cut_point] + second.genotype[cut_point:]
    )
    return offspring

In [201]:
# update population

population = initialize_population()
initialize_fitness(population)

for generation in range(100):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random.random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutation(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = xover(p1, p2)
        offspring.append(o)

    for i in range(len(offspring)):
        offspring[i].fitness = fitness(offspring[i])

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print(population[0].fitness, population[9].genotype)

0.8 [0.342103124762283, 0.3360206749519814, 0.3218762002857357]
0.8 [0.2424440450098644, 0.5357427403110471, 0.2218132146790886]
0.85 [0.22213943927841384, 0.33014893962948916, 0.4477116210920971]
0.85 [0.7739539619370444, 0.5329691817189934, 0.19326448926126555]
0.85 [0.7739539619370444, 0.5329691817189934, 0.19326448926126555]
0.85 [0.375855957054199, 0.612504472777983, 0.011639570167818063]
0.9 [0.375855957054199, 0.612504472777983, 0.19326448926126555]
0.9 [0.375855957054199, 0.612504472777983, 0.19326448926126555]
0.9 [0.12474059768492057, 0.6189824891355988, 0.2876708297762551]
0.9 [0.375855957054199, 0.612504472777983, 0.011639570167818063]
0.9 [0.375855957054199, 0.612504472777983, 0.011639570167818063]
0.9 [0.375855957054199, 0.612504472777983, 0.19326448926126555]
0.95 [0.7739539619370444, 0.19116006226098647, 0.03488597580196929]
0.95 [0.7739539619370444, 0.19116006226098647, 0.03488597580196929]
0.95 [0.7739539619370444, 0.19116006226098647, 0.03488597580196929]
0.95 [0.124